In [1]:
from pdfminer.high_level import extract_text
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_extracter(path):
    text = extract_text(path)
    return text

def text_chunk(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,separators = ["\n\n", "\n", ".", " ", ""])
    chunks = splitter.split_text(text)
    return chunks

In [2]:
import spacy

nlp = spacy.load('en_core_web_sm')

def text_cleaning(chunks):
    cleaned_chunks = []
    for chunk in chunks:
        doc = nlp(chunk)
        cleaned = " ".join([token.text for token in doc if not token.is_stop and not token.is_punct])
        cleaned_chunks.append(cleaned)
    return cleaned_chunks

def lemmatize_text(cleaned_chunks):
    lemma_chunks = []
    for chunk in cleaned_chunks:
        doc = nlp(chunk)
        lemma = " ".join([token.lemma_ for token in doc])
        lemma_chunks.append(lemma)
    return lemma_chunks

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import pickle
import os

def embed_store(lemma_chunks):
  docs = [Document(page_content=chunk) for chunk in lemma_chunks]
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  db = FAISS.from_documents(docs, embeddings)  # FAISS - Facebook ai similarity search (full form for revision)
  
  storage_dir = "/Users/pmanthan/Desktop/tomo.ai/faiss_storage"
  os.makedirs(storage_dir, exist_ok=True)
  db.save_local(storage_dir)
  
  with open("/Users/pmanthan/Desktop/tomo.ai/faiss_storage/faiss_storage.pkl", "wb") as f:
    pickle.dump(embeddings, f)
    
  return db


def load_store():
  storage_dir = "/Users/pmanthan/Desktop/tomo.ai/faiss_storage"
  with open("/Users/pmanthan/Desktop/tomo.ai/faiss_storage/faiss_storage.pkl", "rb") as f:
    embeddings = pickle.load(f)
    
    vector_store = FAISS.load_local(storage_dir, embeddings, allow_dangerous_deserialization=True)
    return vector_store
  
  return vector_store

In [4]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq

def rag_pipeline(): 
   extracted_text = text_extracter("/Users/pmanthan/Desktop/attentionisalluneed.pdf")
   chunked_text = text_chunk(extracted_text)
   cleaned_text = text_cleaning(chunked_text)
   lemma_text = lemmatize_text(cleaned_text)
   embed_text = embed_store(lemma_text)
   return load_store()


In [5]:
llm=ChatGroq(model="llama3-70b-8192",api_key="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")

In [6]:
vector_store = rag_pipeline()
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True,output_key="answer")
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriever,memory=memory,return_source_documents=True)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
/var/folders/gn/l9789lc92mxgrlbrvzzxm2d80000gn/T/ipykernel_59042/2354277832.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the cl

In [7]:
def hybrid_answer(question):
    response = qa_chain.invoke({"question": question})

    if not response["source_documents"]:
        fallback_answer = llm.invoke(question)
        return fallback_answer

    return response["answer"]

def input_output(question):
    hy_answer = hybrid_answer(question)
    print(hy_answer)


input_output("can u summarize the pdf for me")


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}